# Pārraudzītā mašīnmācīšanās — Mobilie tālruņi
**Datu kopa:** `mobile_phones_2000.csv`

Šis notebook demonstrē pilnu klasifikācijas caurplūsmu (supervised learning), lai paredzētu mobilā tālruņa ražotāju (**Brand**) pēc tā tehniskajām specifikācijām.

## 1 — Importi un iestatījumi

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
%matplotlib inline


## 2 — Datu ielāde

In [ ]:

df = pd.read_csv('mobile_phones_2000.csv')
print(df.shape)
display(df.head())


### 2.1 — Pirmsapstrādes setups

In [ ]:

# Mērķa mainīgais
y = df['Brand']

# Pilnībā atmetam 'Model', lai izvairītos no pārāk detalizēta un iespējams noplūstoša atribūta
X = df.drop(columns=['Brand', 'Model'])

# Nosakam kolonu tipus
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print('Skaitliskās kolonnas:', numeric_cols)
print('Kategoriskās kolonnas:', categorical_cols)

# Kolonām ar kameras setup ir daudz unikālu vērtību; saglabājam!

# Preprocessing: skaitliskie -> StandardScaler, kategoriskie -> OneHotEncoder
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)


### 2.2 — Train/Test sadalījums

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
print('Train:', X_train.shape, '  Test:', X_test.shape)


## 3 — Modeļi un rezultāti

### 3.1 — Loģistiskā regresija (baseline)

In [ ]:

logreg_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(max_iter=500, n_jobs=-1, multi_class='multinomial'))
])

logreg_pipeline.fit(X_train, y_train)
y_pred_lr = logreg_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_lr))


### 3.2 — Random Forest + GridSearchCV

In [ ]:

rf = RandomForestClassifier(random_state=42)
param_grid = {
    'clf__n_estimators': [200, 400],
    'clf__max_depth': [None, 20]
}

rf_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', rf)
])

grid = GridSearchCV(
    rf_pipeline, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1
)
grid.fit(X_train, y_train)

print('Labākie parametri:', grid.best_params_)
y_pred_rf = grid.predict(X_test)
print(classification_report(y_test, y_pred_rf))


### 3.3 — Support Vector Machine (rbf)

In [ ]:

svm_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', SVC(kernel='rbf', C=10, gamma='scale'))
])

svm_pipeline.fit(X_train, y_train)
y_pred_svm = svm_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_svm))


### 3.4 — Salīdzinošs kopsavilkums

In [ ]:

from sklearn.metrics import accuracy_score

results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest (best)', 'SVM (RBF)'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_svm)
    ]
})
display(results)


### 3.5 — Konfūzijas matrica labākajam modelim

In [ ]:

best_idx = results['Accuracy'].idxmax()
best_model_name = results.loc[best_idx, 'Model']
best_preds = [y_pred_lr, y_pred_rf, y_pred_svm][best_idx]

cm = confusion_matrix(y_test, best_preds, labels=sorted(y.unique()))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=sorted(y.unique()))
fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
plt.title(f'Confusion matrix — {best_model_name}')
plt.show()


## 4 — Secinājumi

Notebook demonstrē datu priekšapstrādi ar **ColumnTransformer**, trīs klasifikācijas algoritmus un to salīdzinājumu. Rezultāti var atšķirties atkarībā no izvēlētajiem parametriem; iesakāms izpētīt arī tādus modeļus kā Gradient Boosting vai XGBoost, kā arī detalizētāk apskatīt kļūdainās klasifikācijas, lai uzlabotu veiktspēju.